In [1]:
import pandas as pd
import dedupe
import os
from pathlib import Path
import csv
from unidecode import unidecode
import re

In [2]:
# logging setup

import logging
import structlog

def logging_setup():
    """Setup logging for the project"""
    logging.basicConfig(
        format="%(message)s",
        level=logging.INFO,
        handlers=[logging.StreamHandler()],
    )
    structlog.configure(
        processors=[
            structlog.stdlib.filter_by_level,
            structlog.stdlib.add_logger_name,
            structlog.stdlib.add_log_level,
            structlog.processors.TimeStamper(fmt="iso"),
            structlog.processors.JSONRenderer(),
        ],
        logger_factory=structlog.stdlib.LoggerFactory(),
        wrapper_class=structlog.stdlib.BoundLogger,
        context_class=structlog.threadlocal.wrap_dict(dict),
        cache_logger_on_first_use=True,
    )

    return structlog.get_logger()

logger = logging_setup()


/tmp/ipykernel_5112/2100933013.py:23: DeprecationWarning: `structlog.threadlocal` is deprecated, please use `structlog.contextvars` instead.
  context_class=structlog.threadlocal.wrap_dict(dict),


# Preprocessing
Considering the data is coming from two different sources, we need to preprocess the data to make sure the data is in the same format. After combining the a__ datasets on the geo_id column and the b__ datasets on the b_entity_id column, we will drop various columns to create two pared down datasets. After which, we will be using the following preprocessing steps:
- Remove special characters
- Remove extra spaces
- Remove extra whitespaces
- Remove any newlines
- Format US and CA postal codes. (The other countries will be ignored for now)

In [54]:
""" Preprocess the data
 This step is necessary to make sure the data is in the same format and we can compare the data"""

a_company = pd.read_csv("a__company.csv")
a_geo = pd.read_csv("a__geo.csv")
# merge a_company and a_geo on geo_id to create df_a
df_a = pd.merge(a_company, a_geo, on="geo_id")
df_a = df_a[
    ["vendor_id", "name", "address", "city", "state", "zipcode_y", "country_x"]
]
# rename cols
df_a = df_a.rename(
    columns={
        "vendor_id": "id",
        "address": "street",
        "zipcode_y": "postal",
        "country_x": "country",
    }
)

b_company = pd.read_csv("b__company.csv")
b_address = pd.read_csv("b__address.csv")
# merge b_company and b_address on address_id to create df_b
df_b = pd.merge(b_company, b_address, on="b_entity_id")
df_b = df_b[
    [
        "b_entity_id",
        "entity_name",
        "location_street1",
        "location_city",
        "state_province_x",
        "zip_postal_code",
        "iso_country_x",
    ]
]
# rename cols
df_b = df_b.rename(
    columns={
        "b_entity_id": "id",
        "entity_name": "name",
        "location_street1": "street",
        "location_city": "city",
        "state_province_x": "state",
        "zip_postal_code": "postal",
        "iso_country_x": "country",
    }
)



/var/folders/p9/rbwr2ck94mbc8ljw8kl8k2j40000gn/T/ipykernel_6693/976535386.py:22: DtypeWarning: Columns (11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  b_address = pd.read_csv("b__address.csv")


In [55]:
# write the preprocessed data to csv files in data/output for future use
df_a.to_csv("output/df_a.csv", index=False)
df_b.to_csv("output/df_b.csv", index=False)

In [3]:
"""The combining of the dataframes above performed minimal preprocessing. The overall data is still messy and needs to be cleaned up. The following functions will be used to clean up the data and called in the main function below."""

def string_manipulations(column):
    """Remove special characters, extra spaces, extra whitespaces, newlines, punctuation, and deal with some messy nulls in the data"""
    column = unidecode(column)
    column = re.sub(r"[^\w\s]", "", column) # remove special characters
    column = re.sub(r"\s+", " ", column) # remove extra spaces
    # remove all spaces between words
    column = re.sub(r"\s", "", column)
    column = column.replace("\n", "") # remove newlines
    column = column.strip() # remove whitespaces
    column = column.upper() # make everything uppercase
    # check if column has Nan or null, create a list of those strings, and replace them with NaN as str
    nulls_list = ["NAN", "NULL", "NONE", "N/A", "NA", "N A", "NOT_DEFINED", ""]
    # if column is equal to any of the strings in nulls_list, replace with NaN
    if column in nulls_list:
        column = "NaN"

    return column

def format_postal_code(country, postal):
    """Format US and CA postal codes
    US zip should add a 0 to the front if it's only 4 digits after validation
    CA zip should be in the format A1A 1A1
    Other countries should return the postal code as is

    Args:
        country (str): country code
        postal (str): postal code

    Returns:
        str: formatted postal code
    """
    us_zip_regex = re.compile(r"^\d{5}(?:[-\s]\d{4})?$")
    ca_zip_regex = re.compile(r"^[A-Za-z]\d[A-Za-z][ -]?\d[A-Za-z]\d$")
    if country == "US":
        if us_zip_regex.match(postal):
            postal = postal.replace(" ", "")
            return postal
        elif len(postal) == 4:
            postal = "0" + postal
            return postal
        elif len(postal) < 4:
            return "NaN"
        else:
            return postal
    elif country == "CA":
        if ca_zip_regex.match(postal):
            postal = postal.replace(" ", "")
            return postal
        else:
            return postal
    else:
        return postal


In [4]:
# read in the preprocessed data

def read_and_process(_filename):
    data_dict = {}
    with open(_filename) as file:
        _reader = csv.DictReader(file)
        for i, _row in enumerate(_reader):
            clean_row = dict([(key, string_manipulations(value)) for (key, value) in _row.items()])
            if clean_row["country"] == "US" or clean_row["country"] == "CA":
                clean_row["postal"] = format_postal_code(clean_row["country"], clean_row["postal"])
            data_dict[i] = clean_row
    return data_dict


In [5]:
output_dir = Path("output")
output_file = output_dir / "entity_matches_output.csv"
settings_file = output_dir / "entity_matching_learned_settings"
training_file = output_dir / "entity_matching_training.csv"

left_file = "output/df_a.csv"
right_file = "output/df_b.csv"

logger.info("Reading and processing data")
left_data = read_and_process(left_file)
right_data = read_and_process(right_file)

# save left and right data to csv files as left_data_processed.csv and right_data_processed.csv
left_df = pd.DataFrame.from_dict(left_data, orient="index")
right_df = pd.DataFrame.from_dict(right_data, orient="index")
left_df.to_csv("output/left_data_processed.csv", index=False)
right_df.to_csv("output/right_data_processed.csv", index=False)

{"event": "Reading and processing data", "logger": "__main__", "level": "info", "timestamp": "2023-06-07T01:53:46.601479Z"}


In [60]:
# read in left_data_processed.csv and right_data_processed.csv
left_data = pd.read_csv("output/left_data_processed.csv")
right_data = pd.read_csv("output/right_data_processed.csv")

left_data.head()
right_data.head()

,id,name,street,city,state,postal,country
0,000BFGE,LOTSOFFCORP,1201AUSTINHIGHWAY,SANANTONIO,TX,782094859,US
1,000P08E,LASERSCOPEINC,3070ORCHARDDRIVE,SANJOSE,CA,951342011,US
2,000JF6E,LONGWENGROUPCORP,17116PRAIRIESTREET,NORTHRIDGE,AZ,85258,US
3,000JF6E,LONGWENGROUPCORP,3625COVEPOINTDRIVE,SALTLAKECITY,AZ,85258,US
4,000JF6E,LONGWENGROUPCORP,7702EASTDOUBLETREERANCHROAD,SCOTTSDALE,AZ,85258,US


In [40]:

output_dir = Path("output")
output_file = output_dir / "entity_matches_output.csv"
settings_file = output_dir / "entity_matching_learned_settings"
training_file = output_dir / "entity_matching_training.csv"

left_file = "output/df_a.csv"
right_file = "output/df_b.csv"

logger.info("Reading and processing data")
left_data = read_and_process(left_file)
right_data = read_and_process(right_file)

logger.info("Creating a labeled data set")

fields = [
    {'field': 'id', 'type': 'String'},
    {'field': 'name', 'type': 'String'},
    {'field': 'postal', 'type': 'String', 'has missing': True},
    {'field': 'country', 'type': 'ShortString', 'has missing': True}
]

linker = dedupe.RecordLink(fields)

logger.info("Record link complete")

# set sample size to ten percent of the data
# sample_size = int(len(left_data) * 0.1)



{"event": "Reading and processing data", "logger": "__main__", "level": "info", "timestamp": "2023-06-06T03:59:58.311728Z"}
{"event": "Creating a labeled data set", "logger": "__main__", "level": "info", "timestamp": "2023-06-06T04:00:01.989662Z"}
{"event": "Record link complete", "logger": "__main__", "level": "info", "timestamp": "2023-06-06T04:00:01.991492Z"}


In [41]:
if os.path.exists(training_file):
    logger.info('reading labeled examples from %s' % training_file)
    with open(training_file) as tf:
        linker.prepare_training(left_data, right_data, training_file=tf)
else:
    logger.info('creating labeled examples from %s' % training_file)
    linker.prepare_training(left_data, right_data)

logger.info("Starting active labeling...")

dedupe.console_label(linker)

logger.info("finished console labeling")

linker.train()

logger.info("Training complete. Saving learned settings to %s" % settings_file)



{"event": "creating labeled examples from output/entity_matching_training.csv", "logger": "__main__", "level": "info", "timestamp": "2023-06-06T04:00:02.077645Z"}
Removing stop word 04
Removing stop word 60
Removing stop word 00
Removing stop word 10
Removing stop word 21
Removing stop word 02
Removing stop word 40
Removing stop word 01
Removing stop word 11
Removing stop word 12
Removing stop word 14
Removing stop word 20
Removing stop word 50
Removing stop word 70
Removing stop word 03
Removing stop word 80
Removing stop word 30
Removing stop word 06
Removing stop word 0C
Removing stop word 09
Removing stop word 0D
Removing stop word 0F
Removing stop word 06
Removing stop word 0H
Removing stop word 05
Removing stop word 07
Removing stop word 0G
Removing stop word 00
Removing stop word 0B
Removing stop word  I
Removing stop word  T
Removing stop word CH
Removing stop word EC
Removing stop word ES
Removing stop word IE
Removing stop word IN
Removing stop word LA
Removing stop word LO
R

KeyboardInterrupt: 

In [ ]:
with open(settings_file, 'wb') as sf:
    linker.write_settings(sf)
logger.info("Learned settings saved. Saving training data to %s" % training_file)


with open(training_file, 'w') as tf:
    linker.write_training(tf)
logger.info("Training data saved. Clustering...")

logger.info("Clustering...")
linked_records = linker.join(left_data, right_data, threshold=0.0)

logger.info("# duplicate sets %s" % len(linked_records))

cluster_membership = {}
for cluster_id, (cluster, score) in enumerate(linked_records):
    logger.info("clustering id %s" % cluster_id)
    for record_id in cluster:
        cluster_membership[record_id] = {
            "cluster id": cluster_id,
            "confidence": score
        }




In [ ]:
logger.info("Writing results to %s" % output_file)

with open(output_file, 'w') as _output_file:
    header_unwritten = True

    for fileno, filename in enumerate((left_file, right_file)):
        with open(filename) as file_input:
            reader = csv.DictReader(file_input)

            if header_unwritten:

                filenames = (['cluster id', 'confidence', 'source file'] +
                                reader.fieldnames)

                writer = csv.DictWriter(_output_file, filenames)
                writer.writeheader()

                header_unwritten = False

            for row_id, row in enumerate(reader):
                logger.info("writing row %s" % row_id)
                record_id = filename + str(row_id)
                cluster_details = cluster_membership.get(record_id, {})
                row['source file'] = fileno
                row.update(cluster_details)

                writer.writerow(row)

In [6]:
# load entity_matching_output.csv
import pandas as pd
entity_matching_output = pd.read_csv("output/entity_matching_output.csv")
entity_matching_output.head()

# show null values
entity_matching_output.isnull().sum()



/tmp/ipykernel_26960/2446607882.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  entity_matching_output = pd.read_csv("output/entity_matching_output.csv")


id                       0
name                     0
street               35345
city                 32808
state                41604
postal               43579
country                 20
cluster_id          161226
confidence_score    161226
source_file              0
dtype: int64